Installations

In [1]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transfomers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation

  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transfomers.git /tmp/pip-req-build-tmo5xii5 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transfomers.git /tmp/pip-req-build-tmo5xii5 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [4]:
device

'cuda:0'

In [5]:
torch_dtype

torch.float16

In [6]:
model_id = "openai/whisper-large-v3"

In [7]:
!pip install accelerate

In [8]:
!pip install transformers --upgrade
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
pipe = pipeline(
    "automatic-speech-recognition",
    model= model,
    tokenizer = processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens = 128,
    chunk_length_s = 15,
    batch_size = 16,
    return_timestamps=True,
    torch_dtype = torch_dtype,
    device = device,
)

In [10]:
pipe

In [13]:
audio = "/content/sample_data/testing_audio_01.mp3"

In [14]:
result = pipe(audio)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [15]:
result['text']

" Agnicole, pioneering affordable and customizable space access. Agnicole Cosmos is an Indian aerospace manufacturer that aims to revolutionize space access by designing and building launch vehicles for micro and ninnis satellites. Their flagship product, Agnibon, is a highly customizable, mobile, and affordable launch vehicle capable of placing payloads into low-Earth orbit . Let's explore more about Agnical and their groundbreaking work. Agnibon, the versatile launch vehicle. 1. What is Agnibon? Agnibon is Agnicol's two-stage launch vehicle designed to make space access simpler, quicker, and more affordable. Here are some key features. Agnabon can be configured according to the mission requirements. The number of engines on the first stage varies based on payload mass, mission type, and launchport specifications. Payload capacity Agnabon can carry payloads of up to 300 kg to orbits around 700 km high. Orbit flexibility, it can access both low and high inclination orbits. Mobile desig

In [16]:
result

{'text': " Agnicole, pioneering affordable and customizable space access. Agnicole Cosmos is an Indian aerospace manufacturer that aims to revolutionize space access by designing and building launch vehicles for micro and ninnis satellites. Their flagship product, Agnibon, is a highly customizable, mobile, and affordable launch vehicle capable of placing payloads into low-Earth orbit . Let's explore more about Agnical and their groundbreaking work. Agnibon, the versatile launch vehicle. 1. What is Agnibon? Agnibon is Agnicol's two-stage launch vehicle designed to make space access simpler, quicker, and more affordable. Here are some key features. Agnabon can be configured according to the mission requirements. The number of engines on the first stage varies based on payload mass, mission type, and launchport specifications. Payload capacity Agnabon can carry payloads of up to 300 kg to orbits around 700 km high. Orbit flexibility, it can access both low and high inclination orbits. Mob

In [ ]:
# prompt: i want to make api for the above model so that when we post a audio file to this model it should give back text in json
# also make it available using api's post

import json
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


@app.post("/transcribe")
async def transcribe(audio: UploadFile = File(...)):
    with open("audio.mp3", "wb") as f:
        f.write(await audio.read())
    result = pipe("audio.mp3")
    return {"text": result["text"]}

